In [2]:
#Actualizado 19/05/2022 10:50

"""
El siguiente script permite llenar los contenidos y actividades en el portal de schooltrack.
Todo se hará automáticamente.
Los datos del contenido y las actividades deben estar previamente ingresados en un excel.
Recordar omitir fechas en las que no hubo clases.

El bot, por ahora, está automatizado para completar un curso en específico.
Por lo que hace falta correr el bot de cada curso para completar todos los formularios

Cada curso tendrá un bot y el usuario cambiará según el profesor que lo esté administrando.
Una vez hecho esto basta que el profesor de turno complete los contenidos en el excel y que
la persona encargada de aplicar el bot lo haga.
"""


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
import pandas as pd


#hacer que esta parte tome los datos desde un documento de texto
#así no tengo que revelar mis datos cuando lo suba a github


usuario_1 = "usuario_1"
password_jota = "usuario_1"

usuario_2 = "usuario_2"
password_2 = "usuario_2"

usuario_3 = "usuario_3"
password_3 = "usuario_3"


#dict con link para cada curso
link_cursos ={
    '1AMTN':'//*[@id="asig_row0"]/span[2]',
    '1BMTN':'//*[@id="asig_row1"]/span[2]',
    '1CMTN':'//*[@id="asig_row6"]/span[2]',
    '1DMTN':'//*[@id="asig_row7"]/span[2]',
    '1EMTN':'//*[@id="asig_row8"]/span[1]',
    '2AMTN':'//*[@id="asig_row2"]/span[2]',
    '2BMTN':'//*[@id="asig_row3"]/span[2]',
    '2CMTN':'//*[@id="asig_row9"]/span[2]',
    '2DMTN':'//*[@id="asig_row4"]/span[2]',
    '2EMTN':'//*[@id="asig_row10"]/span[2]',
    '3AMTN': '//*[@id="asig_row4"]/span[2]',
    '3BMTN':'//*[@id="asig_row5"]/span[2]',
    '3CMTN':'//*[@id="asig_row11"]/span[2]',
    '3DMTN':'//*[@id="asig_row5"]/span[2]',
    '3EMTN':'//*[@id="asig_row6"]/span[2]',
    '4AN':'//*[@id="asig_row12"]/span[2]',
    '4BN':'//*[@id="asig_row14"]/span[2]',
    '4CN':'//*[@id="asig_row16"]/span[2]',
    '4DN':'//*[@id="asig_row7"]/span[2]',
    '4AT':'//*[@id="asig_row13"]/span[2]',
    '4BT':'//*[@id="asig_row15"]/span[2]',
    '4CT':'//*[@id="asig_row6"]/span[2]',
    '4DT':'//*[@id="asig_row8"]/span[2]'
    }


#-----(Mejora: Crear script de scrapeo de ids)-----
#Ids de formularios por curso:

entradas_1AMTN = [945617, 947051, 953289, 950921, 959755, 963031, 968930, 972307, 975578, 978849]
entradas_1BMTN = [947813, 947814, 953947, 957167, 963683, 966327, 969582, 972967, 976238]
entradas_1CMTN = [947916, 951347, 954605, 957819, 961065, 964341, 966985, 970240, 973627, 976898, 980169]
entradas_1DMTN = [947974, 947975, 953929, 957149, 960389, 963665, 966309, 969564, 972949, 976220, 979491]
entradas_1EMTN = [948122, 948123, 952586, 955852, 959054, 962336, 965638, 968235, 971596, 974884, 978155]

entradas_2AMTN = [948337, 951408, 954666, 957880, 961126, 964402, 967046, 970301, 973688, 976959]
entradas_2BMTN = [948489, 948490, 953990, 957210, 960450, 963726, 966370, 969625, 973010, 976281,979552]
entradas_2CMTN = [948658, 948659, 953323, 956547, 959789, 963065, 968964, 972341, 975612, 978883]
entradas_2DMTN = [948847, 948848, 953308, 956532, 959774, 963050, 968949, 972326, 975597, 978868]
entradas_2EMTN = [948883, 948884, 953963, 957183, 960423, 963699, 966343, 969598, 972983, 976254, 979525]

entradas_3AMTN = [949273, 949274, 952722, 955988, 959190, 962472, 965774, 968371, 971732, 975019, 978290, ]
entradas_3BMTN = [960506, 963782, 966426, 969681, 973066, 976337, 979608]#habia contenido ingresado de antes
entradas_3CMTN = [949400, 951438, 954696, 957910, 961156, 964432, 967077, 970332, 973719, 976990, 980261]

entradas_4AN = [949704, 949705, 952799, 956065, 959267, 962549, 965851, 968448, 971809, 975096, 978367]
entradas_4BN = [950093, 950094, 952784, 956050, 959252, 962534, 965836, 968433, 971794, 975081, 978352]
entradas_4CN = [950521, 950522, 953426, 956650, 959892, 963168, 969067, 972444, 975715, 978986]
entradas_4DN = [952743, 956009, 959211, 962493, 965795, 968392, 971753, 975040, 978311, 981582]
entradas_4AT = [949948, 949949, 953455, 956679, 959921, 963197, 969096, 972473, 975744, 979015]
entradas_4BT = [950223, 950225, 952113, 955377, 958583, 961859, 965153, 967750, 971109, 974396, 977667]
entradas_4CT = [950637, 950638, 953423, 956647, 959889, 963165, 969064, 972441, 975712, 978983]
entradas_4DT = [952077, 955341, 958547, 961823, 965117, 967714, 971073, 974362, 977633, 980904]


#ids de cada google sheet:
sheet_ids = {
    '1eros':"1Er041JYmiaNeSMTVTQLO0nFRpoFmGWb1tSZdbX6vMZQ",
    '2dos': "1mE0W-pIyxDj6BzOQ70kdmlHPX2Z6ulT33iycueO1nE0",
    '3eros': "1QfF1HraB7MMxEOuv6HRkpR7X91gTfpm7zR8HJQupeds",
    '4tos': "1Y3j0J4FIzvsF-bRwcpAYbTb20iXcR_JqSdAtDyAwK4o",
    }

#nombre de cada google sheet:
sheet_names = {
    '1eros':"contenidos_primero_basico",
    '2dos': "contenidos_segundo_basico",
    '3eros':"contenidos_tercero_basico",
    '4tos':"contenidos_cuarto_basico"
    }


In [41]:
#***************Script para primeros básicos***************


#Conexión a google sheet del curso:
sheet_url = f"https://docs.google.com/spreadsheets/d/{sheet_ids['1eros']}/gviz/tq?tqx=out:csv&sheet={sheet_names['1eros']}"

df = pd.read_csv(sheet_url)
#hasta aqui todo ok


#Configuración y conexión a Chrome
url = "http://192.168.3.150/stwa/login.shtml"
options = webdriver.ChromeOptions()
options.add_argument('--kiosk') #pantalla completa
driver_path = "/Users/jota/Documents/Jupyter-notebook/Selenium/chromedriver"
driver = webdriver.Chrome(driver_path, chrome_options=options)
driver.get(url)


#-----(Mejoras: Crear función para ingreso de usuario)-------
#Encontrar campos para llenado
driver.find_element_by_name('user').send_keys(usuario_anna)
driver.find_element_by_name('password').send_keys(password_anna)
driver.find_element(By.XPATH, '//*[@id="enter"]').click()

#Ingresar al libro de clases:
sleep(30)
driver.find_element_by_xpath('//*[@id="a-omitir"]').click()
#-------(Fin función ususario)----------


#-----(Mejoras: Crear función ingreso curso y llenado de formulario)-------
#Probando ingreso a un curso y a los contenidos del curso


#****Cambiar curso manualmente*****
driver.find_element_by_xpath(link_cursos['1BMTN']).click()
#****Cambiar curso manualmente*****

driver.find_element_by_xpath('//*[@id="a-sesiones"]').click()
sleep(5)

#BUG!: LA SIGUIENTE PARTE SÓLO FUNCIONA EN PANTALLA COMPLETA:


#****Cambiar curso manualmente*****
for i in entradas_1BMTN:
#****Cambiar curso manualmente*****
    driver.find_element_by_xpath(f'//*[@id="{i}"]').click()
    #****Cambiar curso manualmente*****
    driver.find_element_by_xpath('//*[@id="contenidos"]').send_keys(df.contenido[entradas_1BMTN.index(i)])
    driver.find_element_by_xpath('//*[@id="actividades"]').send_keys(df.actividad[entradas_1BMTN.index(i)])
    #****Cambiar curso manualmente*****
    sleep(0.5)
#-----(Fin función llenado formulario)--------


<ipython-input-41-44bfb3d41ac6>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, chrome_options=options)
<ipython-input-41-44bfb3d41ac6>:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(driver_path, chrome_options=options)
<ipython-input-41-44bfb3d41ac6>:22: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  driver.find_element_by_name('user').send_keys(usuario_anna)
<ipython-input-41-44bfb3d41ac6>:23: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  driver.find_element_by_name('password').send_keys(password_anna)
<ipython-input-41-44bfb3d41ac6>:28: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="a-omitir"]').click()
<ipython-inpu

In [49]:
#***************Script para segundos básicos***************


#Conexión a google sheet del curso:
sheet_url = f"https://docs.google.com/spreadsheets/d/{sheet_ids['2dos']}/gviz/tq?tqx=out:csv&sheet={sheet_names['2dos']}"

df = pd.read_csv(sheet_url)
#hasta aqui todo ok


#Configuración y conexión a Chrome
url = "http://192.168.3.150/stwa/login.shtml"
options = webdriver.ChromeOptions()
options.add_argument('--kiosk') #pantalla completa
driver_path = "/Users/jota/Documents/Jupyter-notebook/Selenium/chromedriver"
driver = webdriver.Chrome(driver_path, chrome_options=options)
driver.get(url)


#-----(Mejoras: Crear función para ingreso de usuario)-------
#Encontrar campos para llenado
driver.find_element_by_name('user').send_keys(usuario_anna)
driver.find_element_by_name('password').send_keys(password_anna)
driver.find_element(By.XPATH, '//*[@id="enter"]').click()

#Ingresar al libro de clases:
sleep(40)
driver.find_element_by_xpath('//*[@id="a-omitir"]').click()
#-------(Fin función ususario)----------


#-----(Mejoras: Crear función ingreso curso y llenado de formulario)-------
#Probando ingreso a un curso y a los contenidos del curso


#****Cambiar curso manualmente*****
driver.find_element_by_xpath(link_cursos['2DMTN']).click()
#****Cambiar curso manualmente*****

driver.find_element_by_xpath('//*[@id="a-sesiones"]').click()
sleep(5)

#BUG!: LA SIGUIENTE PARTE SÓLO FUNCIONA EN PANTALLA COMPLETA:


#****Cambiar curso manualmente*****
for i in entradas_2DMTN:
#****Cambiar curso manualmente*****
    driver.find_element_by_xpath(f'//*[@id="{i}"]').click()
    #****Cambiar curso manualmente*****
    driver.find_element_by_xpath('//*[@id="contenidos"]').send_keys(df.contenido[entradas_2DMTN.index(i)])
    driver.find_element_by_xpath('//*[@id="actividades"]').send_keys(df.actividad[entradas_2DMTN.index(i)])
    #****Cambiar curso manualmente*****
    sleep(0.5)
#-----(Fin función llenado formulario)--------


<ipython-input-49-2981f0f13745>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, chrome_options=options)
<ipython-input-49-2981f0f13745>:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(driver_path, chrome_options=options)
<ipython-input-49-2981f0f13745>:22: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  driver.find_element_by_name('user').send_keys(usuario_anna)
<ipython-input-49-2981f0f13745>:23: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  driver.find_element_by_name('password').send_keys(password_anna)
<ipython-input-49-2981f0f13745>:28: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="a-omitir"]').click()
<ipython-inpu

In [5]:
#***************Script para terceros básicos***************


#Conexión a google sheet del curso:
sheet_url = f"https://docs.google.com/spreadsheets/d/{sheet_ids['3eros']}/gviz/tq?tqx=out:csv&sheet={sheet_names['3eros']}"

df = pd.read_csv(sheet_url)
#hasta aqui todo ok


#Configuración y conexión a Chrome
url = "http://192.168.3.150/stwa/login.shtml"
options = webdriver.ChromeOptions()
options.add_argument('--kiosk') #pantalla completa
driver_path = "/Users/jota/Documents/Jupyter-notebook/Selenium/chromedriver"
driver = webdriver.Chrome(driver_path, chrome_options=options)
driver.get(url)


#-----(Mejoras: Crear función para ingreso de usuario)-------
#Encontrar campos para llenado
driver.find_element_by_name('user').send_keys(usuario_daniel)
driver.find_element_by_name('password').send_keys(password_daniel)
driver.find_element(By.XPATH, '//*[@id="enter"]').click()

#Ingresar al libro de clases:
sleep(30)
driver.find_element_by_xpath('//*[@id="a-omitir"]').click()
#-------(Fin función ususario)----------


#-----(Mejoras: Crear función ingreso curso y llenado de formulario)-------
#Probando ingreso a un curso y a los contenidos del curso


#****Cambiar curso manualmente*****
driver.find_element_by_xpath(link_cursos['3BMTN']).click()
#****Cambiar curso manualmente*****

driver.find_element_by_xpath('//*[@id="a-sesiones"]').click()
sleep(5)

#BUG!: LA SIGUIENTE PARTE SÓLO FUNCIONA EN PANTALLA COMPLETA:


#****Cambiar curso manualmente*****
for i in entradas_3BMTN:
#****Cambiar curso manualmente*****
    driver.find_element_by_xpath(f'//*[@id="{i}"]').click()
    #****Cambiar curso manualmente*****
    driver.find_element_by_xpath('//*[@id="contenidos"]').send_keys(df.contenido[entradas_3BMTN.index(i)])
    driver.find_element_by_xpath('//*[@id="actividades"]').send_keys(df.actividad[entradas_3BMTN.index(i)])
    #****Cambiar curso manualmente*****
    sleep(0.5)
#-----(Fin función llenado formulario)--------


<ipython-input-5-30e4ec2263aa>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, chrome_options=options)
<ipython-input-5-30e4ec2263aa>:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(driver_path, chrome_options=options)
<ipython-input-5-30e4ec2263aa>:22: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  driver.find_element_by_name('user').send_keys(usuario_daniel)
<ipython-input-5-30e4ec2263aa>:23: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  driver.find_element_by_name('password').send_keys(password_daniel)
<ipython-input-5-30e4ec2263aa>:28: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="a-omitir"]').click()
<ipython-input

In [5]:
#***************Script para cuartos básicos***************


#Conexión a google sheet del curso:
sheet_url = f"https://docs.google.com/spreadsheets/d/{sheet_ids['4tos']}/gviz/tq?tqx=out:csv&sheet={sheet_names['4tos']}"

df = pd.read_csv(sheet_url)
#hasta aqui todo ok


#Configuración y conexión a Chrome
url = "http://192.168.3.150/stwa/login.shtml"
options = webdriver.ChromeOptions()
options.add_argument('--kiosk') #pantalla completa
driver_path = "/Users/jota/Documents/Jupyter-notebook/Selenium/chromedriver"
driver = webdriver.Chrome(driver_path, chrome_options=options)
driver.get(url)


#-----(Mejoras: Crear función para ingreso de usuario)-------
#Encontrar campos para llenado
driver.find_element_by_name('user').send_keys(usuario_jota)
driver.find_element_by_name('password').send_keys(password_jota)
driver.find_element(By.XPATH, '//*[@id="enter"]').click()

#Ingresar al libro de clases:
sleep(60)
driver.find_element_by_xpath('//*[@id="a-omitir"]').click()
#-------(Fin función ususario)----------


#-----(Mejoras: Crear función ingreso curso y llenado de formulario)-------
#Probando ingreso a un curso y a los contenidos del curso


#****Cambiar curso manualmente*****
driver.find_element_by_xpath(link_cursos['4AN']).click()
#****Cambiar curso manualmente*****

driver.find_element_by_xpath('//*[@id="a-sesiones"]').click()
sleep(5)

#BUG!: LA SIGUIENTE PARTE SÓLO FUNCIONA EN PANTALLA COMPLETA:


#****Cambiar curso manualmente*****
for i in entradas_4AN:
#****Cambiar curso manualmente*****
    driver.find_element_by_xpath(f'//*[@id="{i}"]').click()
    #****Cambiar curso manualmente*****
    driver.find_element_by_xpath('//*[@id="contenidos"]').clear()
    driver.find_element_by_xpath('//*[@id="contenidos"]').send_keys(df.contenido[entradas_4AN.index(i)])
    driver.find_element_by_xpath('//*[@id="actividades"]').clear()
    driver.find_element_by_xpath('//*[@id="actividades"]').send_keys(df.actividad[entradas_4AN.index(i)])
    #****Cambiar curso manualmente*****
    sleep(0.5)
#-----(Fin función llenado formulario)--------


<ipython-input-5-4adff463002c>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, chrome_options=options)
<ipython-input-5-4adff463002c>:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(driver_path, chrome_options=options)
<ipython-input-5-4adff463002c>:22: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  driver.find_element_by_name('user').send_keys(usuario_jota)
<ipython-input-5-4adff463002c>:23: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  driver.find_element_by_name('password').send_keys(password_jota)
<ipython-input-5-4adff463002c>:28: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="a-omitir"]').click()
<ipython-input-5-4

In [ ]:
#Para Borrado:
sleep(3)
for i in entradas_list:
    driver.find_element_by_xpath(f'//*[@id="{i}"]').click()
    driver.find_element_by_xpath('//*[@id="contenidos"]').clear()
    driver.find_element_by_xpath('//*[@id="actividades"]').clear()
    sleep(0.5)